<a href="https://colab.research.google.com/github/smj-lab2023/test/blob/main/1_folium_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 한글 폰트 설치

- 참고: https://teddylee777.github.io/colab/colab-korean

In [ ]:
# 구글 colab 사용자용
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

## folium 개요

folium은 `leaflet.js` 자바 스크립트 기반으로 만들어진 인터렉티브한 Python 지도 시각화 라이브러리 입니다.

[관련 문서(Doc) 링크](https://python-visualization.github.io/folium/)

folium을 사용하여 인터랙티브한 지도를 생성하고 **마커를 추가**하여 시각화하거나 **원으로 범위를 표기**하고 `html` 파일로 내보내기 등을 수행할 수 있습니다.

## folium 설치

`pip install folium`으로 라이브러리를 설치할 수 있습니다.
###### 아나콘다 환경설정하면서 미리 설치함

In [ ]:
# !pip install folium

지도 시각화 라이브러리 import 

In [ ]:
import folium

In [ ]:
# 경고 표시하지않기

import warnings

warnings.filterwarnings(action='ignore')

## 기본 좌표 설정(초기위치 설정하기)

`location`에 위도, 경도 정보를 입력하여 입력한 **위,경도 좌표**를 기준으로 지도를 그릴 수 있습니다.

이때 `zoom_start` 정보를 지정하여 **확대**의 정도를 지정할 수 있습니다.

- 참고: `zoom_start`의 범위는 **최대 18** 입니다.

### 구글 지도 실행후 검색위치 위경도 좌표 가져오기
### https://www.google.co.kr/maps/
### 구글맵에서 위치 검색후 마우스 우클릭후 주변검색 메뉴 선택으로 위, 경도좌표 가져오기

In [ ]:
# 위도
latitude = 37.5159

# 경도
longitude = 127.0349

In [ ]:
# 코드입력
m = folium.Map(location=[latitude, longitude],
               zoom_start=17, 
               width=750, 
               height=500
              )
m

## 마커 추가

- `location`: 마커를 추가할 **위도/경도 좌표**를 입력 후 
- `popup`: 표기할 팝업 문구 지정 (마우스 클릭시 표기되는 문구)
- `tooltip`: 표기할 툴팁 지정 (마우스 오버시 표기되는 문구)

마커를 생성 후 기존에 생성된 지도 `m`에 추가합니다.

In [ ]:
# 코드입력
folium.Marker([latitude, longitude],
              popup="공간정보아카데미",
              tooltip="공간정보아카데미 입구").add_to(m)
m

마커에 대한 스타일 변경도 가능합니다. 스타일 변경시 `icon` 파라미터에 `folium.Icon(color=?, icon=?)`을 지정합니다.

In [ ]:
# 코드입력
folium.Marker([latitude, longitude],
              popup="공간정보아카데미",
              tooltip="공간정보아카데미 입구", 
              icon=folium.Icon('red', icon='star'),
             ).add_to(m)
m

1.   `popup`이나 `tooltip`에 다음과 같이 `html 코드`를 삽입하여 이미지를 표기하거나 YouTube 영상도 삽입할 수 있습니다.
2.   YouTube에서 해당 동영상 > 공유 > 퍼가기 > 복사순으로 iframe link 가져오기




In [ ]:
folium.Marker([latitude, longitude],
              popup='<iframe width="560" height="315" src="https://www.youtube.com/embed/YHBX1ESeTKI" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>',
              tooltip="공간정보아카데미 입구").add_to(m)
m

In [ ]:
folium.CircleMarker([latitude, longitude],
                    color='tomato',
                    radius = 150, 
                    tooltip='학동역 상권').add_to(m)
m

## 실습에 필요한 데이터셋

[소상공인 시장 진흥공단 상권정보](https://www.data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15083033)

In [ ]:
# !pip install teddynote -q

In [ ]:
# from teddynote import dataset

# dataset.download('소상공인상권정보')

In [ ]:
import pandas as pd

# 코드입력
df = pd.read_csv('./data/상권정보_서울.csv')
df.head(3)

컬럼 정보를 출력 합니다.

In [ ]:
pd.Series(df.columns)

법정동명이 `학동`, `논현동`, `역삼동` 인 지역만 불러옵니다

In [ ]:
sub_df = df.loc[df['법정동명'].isin(['삼성동', '논현동', '역삼동'])]
sub_df.tail(3)

In [ ]:
sub_df[['위도', '경도', '상호명']]

## 클러스터 생성

In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map(
    location=[latitude, longitude],
    zoom_start=15
)

coords = sub_df[['위도', '경도']]

In [ ]:
# 코드입력
marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(coords['위도'], coords['경도']):
    folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)
m

## 서울 지도에서 행정 구역별 표시

In [ ]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)
seoul_geo

서울 지역의 구별 boundary 시각화

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
)

# 코드입력
folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m

## tiles 옵션 변경을 통해 지도의 테마 변경

지도의 기본 테마(tiles)는 `OpenStreetMap`으로 설정되어 있는데, 이를 변경하여 다른 지도 테마를 적용할 수 있다.

### `Stamen Toner` 적용시

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    # 타일변경
    tiles='Stamen Toner'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m

### `cartodbpositron` 적용시

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    # 타일변경
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m

## 서울시 자치구별 상권정보 시각화

- [소상공인시장진흥공단 상권 정보 데이터셋 다운로드](https://www.data.go.kr/data/15083033/fileData.do)

`seoul` 데이터프레임에 소상공인시장진흥공단에서 제공하는 **서울시 상권정보 csv 파일**을 로드합니다. 

In [ ]:
import pandas as pd

# 서울 상권정보 데이터를 로드합니다
seoul = pd.read_csv('./data/상권정보_서울.csv')

# 필요한 컬럼 정보만 가져옵니다
seoul = seoul[['시군구명', '상권업종대분류명', '상권업종중분류명', '위도', '경도']]
seoul

시각화 모듈을 import 하고 **서울시의 업종별 개수**를 시각화합니다.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
# plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.family'] = 'Malgun Gothic'

서울시의 `상권업종중분류명`에 따른 데이터 개수를 시각화 합니다.

In [ ]:
# 코드입력
plt.figure(figsize=(12, 30))
sns.countplot(y=seoul['상권업종중분류명'], order=seoul['상권업종중분류명'].value_counts().index)
plt.yticks(fontsize=12)
plt.title('서울시 업종별 개수')
plt.show()

**한식 업종이 가장 많은 개수**를 차지합니다. 커피점/카페 업종이 가장 많은 업종일 줄 알았는데요. 한식, 이미용/건강, 종합소매업 다음 4위를 차지하였습니다.

그럼 **커피점/카페** 업종은 자치구별로 **어느 곳에 가장 많이 분포**해 있는지 시각화해 보겠습니다.

In [ ]:
# 코드입력
plt.figure(figsize=(12, 10))
seoul.loc[seoul['상권업종중분류명'] == '커피점/카페'].groupby('시군구명')['상권업종대분류명'].count()\
                                                     .sort_values().plot(kind='barh', color='royalblue')
plt.yticks(fontsize=12)
plt.title('서울시 자치구별 커피점/카페 업종수')
plt.show()

강남구가 가장 많은 수를 차지하고 강북구는 가장 적은 수를 차지하였습니다.

이제 이를 folium 위에 시각화를 해보겠습니다.

In [ ]:
seoul_coffee = seoul.loc[seoul['상권업종중분류명'] == '커피점/카페']
seoul_coffee

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

클러스터를 추가하여 지도위에 시각화 합니다.

In [ ]:
# 코드입력
marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(seoul_coffee['위도'], seoul_coffee['경도']):
    folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)

m

In [ ]:
seoul_group_data = seoul.loc[seoul['상권업종중분류명'] == '커피점/카페'].groupby('시군구명')['상권업종중분류명'].count()
seoul_group_data

`choropleth` 를 사용하여 시각화를 하면 업종 별 개수에 따른 색상의 차이를 두어 시각화를 해줍니다.

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

In [ ]:
# 코드입력
m.choropleth(geo_data=seoul_geo,
             data=seoul_group_data, 
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 커피 업종 수"
            )
m

`bins`를 만들어 **1/4, 2/4, 3/4분위수별**로 구간을 나누어 시각화할 수 있습니다.

In [ ]:
# 코드입력
bins = list(seoul_group_data.quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m.choropleth(geo_data=seoul_geo,
             data=seoul_group_data, 
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 커피 업종 수", 
             bins=bins
            )
m

## Html 파일로 저장

저장은 `save()` 함수로 쉽게 html 파일로 저장할 수 있습니다.

In [ ]:
# 구글 colab 사용자
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
m.save('map.html')

## 참고 (References)

- [Folium 0.12.1 documentation](https://python-visualization.github.io/folium/)
- [folium을 이용하여 서울시 동별 인구 수 시각화하기](https://d-hyeon.tistory.com/2)
- [지도 핸들링_folium](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=jymoon1115&logNo=221392797679)